In [22]:
'''VGG11/13/16/19 in Pytorch.'''
import torch
import torch.nn as nn


cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Linear(512, 10)

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)


def test():
    net = VGG('VGG11')
    x = torch.randn(2,3,32,32)
    y = net(x)
    print(y.size())

# test()

In [23]:
from torch.optim.lr_scheduler import _LRScheduler
# from .optimizer import Optimizer

class randomLR(_LRScheduler):


    def __init__(self, optimizer, base_lr, max_lr, step_size=100, last_epoch=-1):
        
        self.step_size = step_size    
        self.max_lrs = self._format_param('max_lr', optimizer, max_lr)
        super(randomLR, self).__init__(optimizer, last_epoch)
        self.base_lrs = self._format_param('base_lr', optimizer, base_lr)
        
        
    def _format_param(self, name, optimizer, param):
        """Return correctly formatted lr/momentum for each param group."""
        if isinstance(param, (list, tuple)):
            if len(param) != len(optimizer.param_groups):
                raise ValueError("expected {} values for {}, got {}".format(
                    len(optimizer.param_groups), name, len(param)))
            return param
        else:
            return [param] * len(optimizer.param_groups)

    def get_lr(self):
        if not self._get_lr_called_within_step:
            warnings.warn("To get the last learning rate computed by the scheduler, "
                          "please use `get_last_lr()`.", UserWarning)

        if (self.last_epoch == 0) or (self.last_epoch % self.step_size == 0):
            lrs = []
            for base_lr, max_lr in zip(self.base_lrs, self.max_lrs):
                lrs.append(np.random.uniform(base_lr, max_lr))
            return lrs
                
        return [group['lr'] for group in self.optimizer.param_groups]

In [24]:
from torch.optim.lr_scheduler import _LRScheduler
# from .optimizer import Optimizer

class MCLR(_LRScheduler):


    def __init__(self, optimizer, base_lr, max_lr, p=0.9, step_size=100, last_epoch=-1):
        
        self.step_size = step_size    
        self.max_lrs = self._format_param('max_lr', optimizer, max_lr)
#         self.base_lrs = self._format_param('base_lr', optimizer, base_lr)
        super(MCLR, self).__init__(optimizer, last_epoch)
        self.base_lrs = self._format_param('base_lr', optimizer, base_lr)
        self.onestep = (max_lr-base_lr)/step_size
        self.p = p
        self.flag = 0
        
        
    def _format_param(self, name, optimizer, param):
        """Return correctly formatted lr/momentum for each param group."""
        if isinstance(param, (list, tuple)):
            if len(param) != len(optimizer.param_groups):
                raise ValueError("expected {} values for {}, got {}".format(
                    len(optimizer.param_groups), name, len(param)))
            return param
        else:
            return [param] * len(optimizer.param_groups)

    def get_lr(self):
        if not self._get_lr_called_within_step:
            warnings.warn("To get the last learning rate computed by the scheduler, "
                          "please use `get_last_lr()`.", UserWarning)
            

            
        if self.last_epoch == 0:
            lrs = []
            for base_lr in self.base_lrs:
                lrs.append(base_lr)

            return lrs
        
            
        elif self.last_epoch == 1:
            lrs = []
            for base_lr in self.base_lrs:
                lrs.append(base_lr+self.onestep)
                
            return lrs
                
        else:
            
            lrs = []
            last_lr = optimizer.param_groups[0]['lr']
            
            
            if last_lr >= self.max_lrs[0]:
                self.p = min(self.p, 1-self.p)
                for max_lr in self.max_lrs:
                    lrs.append(max_lr-self.onestep)
                
            elif last_lr <= self.base_lrs[0]:
                self.p = max(self.p, 1-self.p)
                for base_lr in self.base_lrs:
                    lrs.append(base_lr+self.onestep)
                    
            else:
                tp = np.random.uniform(0,1)
                
                if tp <= self.p:
                    temp_lr = last_lr + self.onestep

                    for max_lr in self.max_lrs:
                        if temp_lr >= max_lr:
                            lrs.append(max_lr)
                        else:
                            lrs.append(temp_lr)


                        
                else:
                    temp_lr = last_lr - self.onestep

                    for base_lr in self.base_lrs:
                        
                        if temp_lr <= base_lr:
                            lrs.append(base_lr)
                        else:
                            lrs.append(temp_lr)

                        
            return lrs
                
            
        

#         if (self.last_epoch == 0) or (self.last_epoch % self.step_size == 0):
#             lrs = []
#             p = np.random.uniform(0,1)
#             for base_lr, max_lr in zip(self.base_lrs, self.max_lrs):
#                 p = np.random.uniform(0,1)
#                 lr = optimizer.param_groups[0]['lr']
#                 threshold = (lr-base_lr)/(max_lr-base_lr)
#                 if p>= threshold:
#                     lrs.append(np.random.uniform(lr, max_lr))
#                 else:
#                     lrs.append(np.random.uniform(base_lr, lr))
#             return lrs
                
#         return [group['lr'] for group in self.optimizer.param_groups]


In [25]:
# from scipy.stats import levy_stable
import numpy as np
import torch
import math
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch import optim
import os

PATH_base = './VGG-CIFAR10-mc-umut'

try:
    os.mkdir(PATH_base)
except OSError as exc:
    pass

In [26]:
batch_size= 10
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
test_loader = DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

lr_list = [0.005, 0.01, 0.015, 0.02]
# lr_list = [0.02]
delta = 0.004
epoch = 500
p=0.6
lr_unif = []

Files already downloaded and verified
Files already downloaded and verified


In [27]:
torch.manual_seed(0)
for i in range(len(lr_list)):
#     lrs_tmp = []
    learning_rate = lr_list[i]
    trainErrorList=[]
    trainAccList=[]
    
    PATH = PATH_base + '/LR' + '{}'.format(i)
    try:
        os.mkdir(PATH)
    except OSError as exc:
        pass
    
                
    model = VGG('VGG11')
    if torch.cuda.is_available():
            model = model.cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    lr_scheduler = randomLR(optimizer,base_lr=learning_rate-delta,max_lr=learning_rate+delta)
    lr_scheduler = MCLR(optimizer,base_lr=learning_rate-delta,max_lr=learning_rate+delta,
                                                     p=p,step_size=100)
    iter_count = 0
    iter_name = 1
    for l in range(epoch):
        train_acc=0
        for data in train_loader:
#             lrs_tmp.append(optimizer.param_groups[0]['lr'])
            img, label = data
            # img=img.view(img.size(0),-1)
            if torch.cuda.is_available():
                img = img.cuda()
                label = label.cuda()
            else:
                img = Variable(img)
                label = Variable(label)
            out = model(img)
            loss = criterion(out, label)
            print_loss = loss.data.item()
            _, pred = torch.max(out.data, 1)
            train_acc += pred.eq(label.view_as(pred)).sum().item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            lr_scheduler.step()
            iter_count += 1
            if iter_count > 300000:
                break
            if iter_count > 299900:
                tmp_path = PATH + '/model' + '{}'.format(iter_name) +'.pth'
                torch.save(model, tmp_path)
                iter_name += 1
                
#         lrs_tmp.append(optimizer.param_groups[0]['lr'])
        if iter_count > 300000:
            break
        trainErrorList.append(loss.data.item())
        trainAccList.append(train_acc/50000)
    print(learning_rate)
    print(train_acc/50000)
#     lr_mc.append(lrs_tmp)

0.005
0.0002
0.01
0.0002
0.015
0.0002
0.02
0.0002


In [28]:
trainAccList

[0.5238,
 0.71114,
 0.77572,
 0.82612,
 0.85922,
 0.8892,
 0.91446,
 0.9306,
 0.94454,
 0.95636,
 0.96456,
 0.97096,
 0.97754,
 0.98182,
 0.98614,
 0.98802,
 0.9864,
 0.98926,
 0.99098,
 0.99294,
 0.99404,
 0.99464,
 0.99468,
 0.99474,
 0.99572,
 0.99612,
 0.99548,
 0.99534,
 0.9967,
 0.99708,
 0.99596,
 0.99686,
 0.99826,
 0.997,
 0.99726,
 0.9977,
 0.99744,
 0.9985,
 0.99872,
 0.99768,
 0.99654,
 0.9987,
 0.99824,
 0.9985,
 0.99924,
 0.9993,
 0.99914,
 0.99962,
 0.99914,
 0.99878,
 0.99842,
 0.9981,
 0.99928,
 0.99938,
 0.99958,
 0.99948,
 0.99796,
 0.99898,
 0.99892,
 0.99832]